In [1]:
#using Imbalance
using Random
using Distances
using Imbalance: precompute_value_encodings, precompute_mvdm_distances, ValueDifference

In [3]:
import Pkg;
Pkg.add("PyCall")
Pkg.build("PyCall")
using Conda
using PyCall
#pyimport_conda("imblearn", "imbalanced-learn")
Conda.add("imbalanced-learn")
np = pyimport("numpy")
imblearn = pyimport("imblearn")
fromsklearn = pyimport("sklearn.preprocessing")

   Resolving package versions...


  No Changes to `~/Documents/GitHub/Imbalance.jl/Project.toml`
  No Changes to `~/Documents/GitHub/Imbalance.jl/Manifest.toml`


    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/8c86e48c0db1564a1d49548d3515ced5d604c408/build.log`


    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/43d304ac6f0354755f1d60730ece8c499980f7ba/build.log`


┌ Info: Running `conda install -y imbalanced-learn` in root environment
└ @ Conda /Users/essam/.julia/packages/Conda/2lg2O/src/Conda.jl:127


done
Solving environment: ...working... 

done



# All requested packages already installed.



PyObject <module 'sklearn.preprocessing' from '/Users/essam/.julia/conda/3/aarch64/lib/python3.10/site-packages/sklearn/preprocessing/__init__.py'>

In [4]:
# Set the dimensions of the matrix
rows = 10000
cols = 8

# Generate a random matrix X and vector y in Python
X = np.random.randint(1, 11, size=(rows, cols))
y = np.random.randint(1, 6, size=rows)

10000-element Vector{Int64}:
 1
 2
 1
 3
 2
 2
 3
 1
 3
 5
 ⋮
 5
 1
 2
 4
 1
 2
 1
 1
 2

In [5]:
mvdm_encoder_python = imblearn.metrics.pairwise.ValueDifferenceMetric().fit(X, y)
mvdm_encoder_python.pairwise(X)

10000×10000 Matrix{Float64}:
 0.0        0.0406474  0.024525   …  0.042967   0.0215846  0.0393755
 0.0406474  0.0        0.0512151     0.0587939  0.0344256  0.0451652
 0.024525   0.0512151  0.0           0.0453854  0.0368614  0.0313901
 0.0325457  0.028312   0.0423967     0.0455556  0.0317445  0.0330468
 0.0442751  0.0520507  0.0294205     0.0276812  0.0559898  0.0293175
 0.0377418  0.0422198  0.04681    …  0.0297435  0.0387461  0.0331488
 0.0399682  0.0575378  0.0394138     0.0460603  0.0489323  0.0596326
 0.0337098  0.0531134  0.0255758     0.0675065  0.0375791  0.0470832
 0.0375253  0.071699   0.03587       0.0505525  0.0545017  0.0300255
 0.0273793  0.0244688  0.0299856     0.0441514  0.0312811  0.0352789
 ⋮                                ⋱                        
 0.0194097  0.0762089  0.0347332     0.0413007  0.0418607  0.0375026
 0.0455924  0.0595245  0.0385077     0.0640884  0.0438537  0.0571864
 0.0344415  0.057619   0.0494754     0.0521037  0.0504505  0.0445732
 0.0436356  0.

In [ ]:
mvdm_encoder, num_cats_per_column = precompute_value_encodings(X, y)
all_pairwise_mvdm = precompute_mvdm_distances(mvdm_encoder, num_cats_per_column)
V = ValueDifference(all_pairwise_mvdm)
D = pairwise(V, X, X, dims=1)

In [ ]:
using Distances
using LoopVectorization

struct MyMetric <: Metric end

function Distances.evaluate(dist::MyMetric, a::AbstractVector, b::AbstractVector)
    sum(abs.(a - b))
end

m = MyMetric()


function pairwise_no_optimization(dist_fun, metric, X)
    n = size(X, 2)
    D = zeros(n, n)
    for i in 1:n
        for j in i+1:n
            D[i, j] = dist_fun(metric, X[:, i], X[:, j])
            D[j, i] = D[i, j]
        end
    end
    return D
end

function pairwise_with_optimization(dist_fun, metric, X)
    n = size(X, 2)
    D = zeros(n, n)

    @inbounds @simd for i in 1:n
        @inbounds @simd for j in i+1:n
            D[i, j] = dist_fun(metric, X[:, i], X[:, j])
            D[j, i] = D[i, j]
        end
    end
    return D
end


using Base.Threads

function pairwise_with_threads(dist_fun, m, X)
    n = size(X, 2)
    D = zeros(n, n)
    
    Threads.@threads for i in 1:n
        for j in i+1:n
            D[i, j] = dist_fun(m, X[:, i], X[:, j])
            D[j, i] = D[i, j]
        end
    end
    
    return D
end

function pairwise_with_turbo(dist_fun, metric, X)
    n = size(X, 2)
    D = zeros(n, n)

    @turbo for i in 1:n
        for j in i+1:n
            D[i, j] = dist_fun(metric, X[:, i], X[:, j])
            D[j, i] = D[i, j]
        end
    end
    return D
end

X = rand(11, 14000)


# Warm up the functions
#pairwise_no_optimization(Distances.evaluate, m, X)
#pairwise_with_optimization(Distances.evaluate, m, X)
#pairwise_with_threads(Distances.evaluate, m, X)

# Measure execution time for the non-optimized version
@time pairwise_no_optimization(Distances.evaluate, m, X)

# Measure execution time for the optimized version
#@time pairwise_with_optimization(Distances.evaluate, m, X)

# Measure 
#@time pairwise_with_threads(Distances.evaluate, m, X)

#Measure speed with turbo
@time pairwise_with_turbo(Distances.evaluate, m, X)